In [1]:
# Jason Qin
# 05/05/21

# Organize VDJdb TCR-antigen data

Organize data by:
1) TCR chain
2) Species
3) Unique antigens

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random

from scipy.spatial.distance import cdist, hamming
import scipy.stats
import sklearn.metrics

from tape.datasets import FastaDataset, pad_sequences
from tape.tokenizers import TAPETokenizer

In [3]:
# relevant data files
data_dir = '/group/ag_cmb/scratch/jqin/projects/tape/data/tcr'
data_file = data_dir + '/full_data.tsv'
data = pd.read_csv(data_file, sep='\t')
data = data.sort_values('complex.id')

### Organize Sequences by Species

Find all CDR3 sequences for each species

In [4]:
species = ['HomoSapiens', 'MusMusculus', 'MacacaMulatta']
species_cdr3_seq = {} # dict mapping species to all CDR3 sequences that come from it
for sp in species:
    species_cdr3_seq[sp] = set(data[data['Species'] == sp]['CDR3'].tolist())

In [5]:
# write the genes to file
for sp in species:
    with open('{}/{}_cdr3.fasta'.format(data_dir, sp), 'w') as f:
        for i, seq in enumerate(species_cdr3_seq[sp]):
            f.write('>SEQ_{}\n'.format(i))
            f.write('{}\n'.format(seq))

In [6]:
human_cdr3 = species_cdr3_seq['HomoSapiens']
macaque_cdr3 = species_cdr3_seq['MacacaMulatta']
mouse_cdr3 = species_cdr3_seq['MusMusculus']

In [7]:
# are there any CDR3 sequences common between the three species?
print('Human and macaque CDR3 overlap: {}'.format(human_cdr3.intersection(macaque_cdr3)))
print('Human and mouse CDR3 overlap: {}'.format(human_cdr3.intersection(mouse_cdr3)))
print('Mouse and macaque CDR3 overlap: {}'.format(mouse_cdr3.intersection(macaque_cdr3)))

Human and macaque CDR3 overlap: set()
Human and mouse CDR3 overlap: {'CASSSGGEQYF', 'CASSLDRGEQYF', 'CAYSGGSNYKLTF', 'CAMRHSNYQLIW', 'CATGSNYQLIW', 'CAVRPGNTGKLIF', 'CAMREGSGGSNYKLTF', 'CAVNSNYQLIW', 'CASSPGQGYEQYF', 'CASSLRGAYEQYF', 'CASRDTYEQYF', 'CAALNSGGSNYKLTF', 'CAASASGGSNYKLTF', 'CASSLGLYEQYF', 'CASSEGGPYEQYF', 'CAVSMDSNYQLIW', 'CASSYSYEQYF', 'CASSFGREQYF', 'CAVDSNYQLIW', 'CAVMDSNYQLIW', 'CAASGNYQLIW', 'CAVSLDSNYQLIW', 'CASRQGYEQYF', 'CALIGGSNYKLTF', 'CASSLGGYEQYF', 'CASSLGDEQYF', 'CASSEQGYEQYF', 'CASSLSGYEQYF', 'CASSLSGAYEQYF', 'CASSESYEQYF', 'CAVSEGNYQLIW', 'CAASMDSNYQLIW', 'CAALDSNYQLIW', 'CVVQDSNYQLIW', 'CASSDAGTGDYEQYF', 'CAVGSNYQLIW', 'CAVSGGDSNYQLIW', 'CASSFDSYEQYF'}
Mouse and macaque CDR3 overlap: set()


In [8]:
print('Number of CDR3 sequences shared between human and mouse: {}'.format(len(human_cdr3.intersection(mouse_cdr3))))
print('Number of unique mouse CDR3 sequences: {}'.format(len(mouse_cdr3)))
print('Number of unique human CDR3 sequences: {}'.format(len(human_cdr3)))

Number of CDR3 sequences shared between human and mouse: 38
Number of unique mouse CDR3 sequences: 3197
Number of unique human CDR3 sequences: 52897


Find all antigen sequences for each species

In [9]:
# want to do: do any species have the same antigens? If so, may need to
# be careful about separating datasets down the line
species_antigen_seq = {}
for sp in species:
    species_antigen_seq[sp] = set(data[data['Species'] == sp]['Epitope'].tolist())

In [10]:
# write the antigens to file
for sp in species:
    with open('{}/{}_antigen.fasta'.format(data_dir, sp), 'w') as f:
        for i, seq in enumerate(species_antigen_seq[sp]):
            f.write('>SEQ_{}\n'.format(i))
            f.write('{}\n'.format(seq))

In [11]:
human_antigens = species_antigen_seq['HomoSapiens']
macaque_antigens = species_antigen_seq['MacacaMulatta']
mouse_antigens = species_antigen_seq['MusMusculus']

In [12]:
# are any antigens common between the three species?
print('Human and macaque antigen overlap: {}'.format(human_antigens.intersection(macaque_antigens)))
print('Human and mouse antigen overlap: {}'.format(human_antigens.intersection(mouse_antigens)))
print('Mouse and macaque antigen overlap: {}'.format(mouse_antigens.intersection(macaque_antigens)))

Human and macaque antigen overlap: set()
Human and mouse antigen overlap: set()
Mouse and macaque antigen overlap: set()


Takeaways:
- All antigens are unique between different species
- Most TCR sequences are unique between different species
- But: some TCR sequences are shared between mouse and human, despite the fact that no antigen sequences are shared. This means that the same TCR sequence binds to different antigens depending on the species

### Organize Sequence by TRA vs TRB Chain

In [13]:
chains = ['TRA', 'TRB']
chain_cdr3_seq = {} # dict mapping TRA/TRB to all CDR3 sequences that come from it
for chain in chains:
    chain_cdr3_seq[chain] = set(data[data['Gene'] == chain]['CDR3'].tolist())

In [14]:
# write the genes to file
for chain in chains:
    with open('{}/{}_cdr3.fasta'.format(data_dir, chain), 'w') as f:
        for i, seq in enumerate(chain_cdr3_seq[chain]):
            f.write('>SEQ_{}\n'.format(i))
            f.write('{}\n'.format(seq))

In [15]:
# are there any CDR3 sequences common between the two genes?
tra_cdr3 = chain_cdr3_seq['TRA']
trb_cdr3 = chain_cdr3_seq['TRB']

In [16]:
print('TRA and TRB CDR3 overlap: {}'.format(tra_cdr3.intersection(trb_cdr3)))
print('TRA CDR3 sequences: {}'.format(len(tra_cdr3)))
print('TRB CDR3 sequences: {}'.format(len(trb_cdr3)))

TRA and TRB CDR3 overlap: {'CASSSVNEQYF', 'CASSDSRGTEAFF'}
TRA CDR3 sequences: 22352
TRB CDR3 sequences: 34900


### Organize data by unique antigen

In [17]:
# keep track of counts and locations of each antigen
antigens = list(data['Epitope'])
antigen_loc_dict = {}
antigen_count_dict = {}
for i, antigen in enumerate(antigens):
    if antigen not in antigen_loc_dict.keys():
        antigen_loc_dict[antigen] = [i]
        antigen_count_dict[antigen] = 0
    else:
        antigen_loc_dict[antigen] += [i]
        antigen_count_dict[antigen] += 1

In [18]:
# find all antigens with same copy number in data
count_antigen_dict = {}
for antigen, count in antigen_count_dict.items():
    if count not in count_antigen_dict.keys():
        count_antigen_dict[count] = [antigen]
    else:
        count_antigen_dict[count] += [antigen]

In [19]:
# split data into 90/5/5 train/valid/test
total_counts = data.shape[0]
data_counts = {'train': int(total_counts * 0.9), 
               'valid': int(total_counts * 0.05), 
               'test': int(total_counts * 0.05)+1}

In [20]:
# keep track of antigen sequences for train/valid/test sets
antigen_data_split_lists = {'train': set(),
                            'valid': set(),
                            'test': set()}
current_counts = {'train': 0, 
                  'valid': 0, 
                  'test': 0}

for count, antigens in count_antigen_dict.items():
    i = 0
    for antigen in antigens:
        if (i%3 == 0) and (current_counts['train'] < data_counts['train']*0.9):
            antigen_data_split_lists['train'].add(antigen)
            current_counts['train'] += count
        elif (i%3 == 1) and (current_counts['valid'] < data_counts['valid']*0.9):
            antigen_data_split_lists['valid'].add(antigen)
            current_counts['valid'] += count
        else:
            if current_counts['test'] < data_counts['test']:
                antigen_data_split_lists['test'].add(antigen)
                current_counts['test'] += count
            else:
                antigen_data_split_lists['valid'].add(antigen)
                current_counts['valid'] += count
        i += 1

In [21]:
# find indices in data for antigens in train/valid/test sets
antigen_data_split_idx = {'train': [],
                          'valid': [],
                          'test': []}

for data_split, antigens in antigen_data_split_lists.items():
    for antigen in antigens:
        antigen_data_split_idx[data_split] += antigen_loc_dict[antigen]

In [22]:
# confirm that the antigens were uniquely partitioned in to train/valid/test
antigen_data_split_train_seqs = set()
antigen_data_split_valid_seqs = set()
antigen_data_split_test_seqs = set()
antigens = list(data['Epitope'])

for idx in antigen_data_split_idx['train']:
    antigen_data_split_train_seqs.add(antigens[idx])

for idx in antigen_data_split_idx['valid']:
    antigen_data_split_valid_seqs.add(antigens[idx])

for idx in antigen_data_split_idx['test']:
    antigen_data_split_test_seqs.add(antigens[idx])
    
print(antigen_data_split_train_seqs.intersection(antigen_data_split_valid_seqs))
print(antigen_data_split_train_seqs.intersection(antigen_data_split_test_seqs))
print(antigen_data_split_valid_seqs.intersection(antigen_data_split_test_seqs))

set()
set()
set()


In [23]:
# write antigens and CDR3 sequences to file
for data_split, idx in antigen_data_split_idx.items():
    antigen_seq = list(data['Epitope'])
    with open('{}/antigen_unique_{}.fasta'.format(data_dir, data_split), 'w') as f:
        for i in idx:
            f.write('>SEQ_{}\n'.format(i))
            f.write('{}\n'.format(antigen_seq[i]))
    
    cdr3_seq = list(data['CDR3'])
    with open('{}/cdr3_unique_{}.fasta'.format(data_dir, data_split), 'w') as f:
        for i in idx:
            f.write('>SEQ_{}\n'.format(i))
            f.write('{}\n'.format(cdr3_seq[i]))

### Organize data by pairs of TRA and TRB CDR3 sequences

In [24]:
# find all TCR CDR3 chains that have both TRA and TRB information
num_seqs = np.max(data['complex.id']) + 1
tra_cdr3_seqs = num_seqs * ['']
trb_cdr3_seqs = num_seqs * ['']
antigen_seqs = num_seqs * ['']

for i, chain, cdr3_seq, antigen_seq in zip(list(data['complex.id']), list(data['Gene']), list(data['CDR3']), list(data['Epitope'])):
    if chain == 'TRA':
        tra_cdr3_seqs[i] = cdr3_seq
    else:
        trb_cdr3_seqs[i] = cdr3_seq
    antigen_seqs[i] = antigen_seq

In [25]:
# check number of entries for each 'complex.id' of data
complex_id_counts = num_seqs * [0]
for i in data['complex.id']:
    complex_id_counts[i] += 1
    
print(np.unique(complex_id_counts, return_counts=True))

(array([    0,     2, 27938]), array([    8, 24869,     1]))


In [26]:
split_order = random.sample(range(num_seqs), k=num_seqs)
train_split = int(num_seqs*0.9)
valid_split = int(num_seqs*0.95)

train_idx = split_order[:train_split]
valid_idx = split_order[train_split:valid_split]
test_idx = split_order[valid_split:]

split_idx = {'train': train_idx, 'valid': valid_idx, 'test': test_idx}

In [27]:
for split, idx in split_idx.items():
    with open('{}/tra_cdr3_paired_{}.fasta'.format(data_dir, split), 'w') as f:
        for i in idx:
            f.write('>SEQ_{}\n'.format(i))
            f.write('{}\n'.format(tra_cdr3_seqs[i]))
        
    with open('{}/trb_cdr3_paired_{}.fasta'.format(data_dir, split), 'w') as f:
        for i in idx:
            f.write('>SEQ_{}\n'.format(i))
            f.write('{}\n'.format(trb_cdr3_seqs[i]))
            
    with open('{}/antigen_paired_{}.fasta'.format(data_dir, split), 'w') as f:
        for i in idx:
            f.write('>SEQ_{}\n'.format(i))
            f.write('{}\n'.format(antigen_seqs[i]))

### Organize data by pairs of TRA and TRB CDR3 sequences AND unique antigens

In [28]:
# use all data with from 'complex.id' > 1
# separate into train, valid, test sets with unique antigens in each
tra_cdr3_seqs_split = {'train': [], 'valid': [], 'test': []}
trb_cdr3_seqs_split = {'train': [], 'valid': [], 'test': []}
antigen_seqs_split = {'train': [], 'valid': [], 'test': []}

for i in range(1, num_seqs):
    if (tra_cdr3_seqs[i] == '') or (trb_cdr3_seqs[i] == ''):
        continue
    if (antigen_seqs[i] in antigen_data_split_lists['train']):
        key = 'train'
    elif (antigen_seqs[i] in antigen_data_split_lists['valid']):
        key = 'valid'
    elif (antigen_seqs[i] in antigen_data_split_lists['test']):
        key = 'test'
    else:
        raise ValueError('Antigen must be in one of {train, valid, test} set.')
    tra_cdr3_seqs_split[key] += [tra_cdr3_seqs[i]]
    trb_cdr3_seqs_split[key] += [trb_cdr3_seqs[i]]
    antigen_seqs_split[key] += [antigen_seqs[i]]

In [29]:
for split, seqs in tra_cdr3_seqs_split.items():
    with open('{}/tra_cdr3_paired_unique_{}.fasta'.format(data_dir, split), 'w') as f:
        for i, seq in enumerate(seqs):
            f.write('>SEQ_{}\n'.format(i))
            f.write('{}\n'.format(seq))
        
for split, seqs in trb_cdr3_seqs_split.items():
    with open('{}/trb_cdr3_paired_unique_{}.fasta'.format(data_dir, split), 'w') as f:
        for i, seq in enumerate(seqs):
            f.write('>SEQ_{}\n'.format(i))
            f.write('{}\n'.format(seq))
            
for split, seqs in antigen_seqs_split.items():
    with open('{}/antigen_paired_unique_{}.fasta'.format(data_dir, split), 'w') as f:
        for i, seq in enumerate(seqs):
            f.write('>SEQ_{}\n'.format(i))
            f.write('{}\n'.format(seq))

In [30]:
data

,complex.id,Gene,CDR3,V,J,Species,MHC A,MHC B,MHC class,Epitope,Epitope gene,Epitope species,Reference,Method,Meta,CDR3fix,Score
62277,0,TRB,CSARGEAITEKLFF,TRBV20-1*01,TRBJ1-4*01,HomoSapiens,HLA-A*02:01,B2M,MHCI,GLCTLVAML,BMLF1,EBV,PMID:24512815,"{""frequency"": ""43/60"", ""identification"": ""anti...","{""cell.subset"": ""CD8"", ""clone.id"": """", ""donor....","{""cdr3"": ""CSARGEAITEKLFF"", ""cdr3_old"": ""CSARGE...",3
18631,0,TRB,CASSLGWLNSDYTF,TRBV16*01,TRBJ1-2*01,MusMusculus,H-2Kb,B2M,MHCI,RALEYKNL,IE3,MCMV,PMID:31818981,"{""frequency"": ""3/33"", ""identification"": ""tetra...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSLGWLNSDYTF"", ""cdr3_old"": ""CASSLG...",1
18630,0,TRB,CASRRQLSNERLFF,TRBV16*01,TRBJ1-4*01,MusMusculus,H-2Kb,B2M,MHCI,RALEYKNL,IE3,MCMV,PMID:31818981,"{""frequency"": ""3/33"", ""identification"": ""tetra...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASRRQLSNERLFF"", ""cdr3_old"": ""CASRRQ...",1
18629,0,TRB,CASSLDSGDTEVFF,TRBV16*01,TRBJ1-1*01,MusMusculus,H-2Kb,B2M,MHCI,RALEYKNL,IE3,MCMV,PMID:31818981,"{""frequency"": ""6/33"", ""identification"": ""tetra...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSLDSGDTEVFF"", ""cdr3_old"": ""CASSLD...",1
18628,0,TRB,CASSLDAGNQAPLF,TRBV16*01,TRBJ1-5*01,MusMusculus,H-2Kb,B2M,MHCI,RALEYKNL,IE3,MCMV,PMID:31818981,"{""frequency"": ""2/84"", ""identification"": ""tetra...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSLDAGNQAPLF"", ""cdr3_old"": ""CASSLD...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74805,24875,TRB,CAWGTLATEQYF,TRBV28*01,TRBJ2-7*01,HomoSapiens,HLA-A*02,B2M,MHCI,RMFPNAPYL,WT1,HomoSapiens,PMID:23637823,"{""frequency"": ""1/4"", ""identification"": ""multim...","{""cell.subset"": ""CD8+"", ""clone.id"": ""TCR5A"", ""...","{""cdr3"": ""CAWGTLATEQYF"", ""cdr3_old"": ""CAWGTLAT...",3
74806,24876,TRA,CAMSLYYGGSQGNLIF,TRAV12-3*01,TRAJ42*01,HomoSapiens,HLA-A*02,B2M,MHCI,KIFGSLAFL,ERBB2,HomoSapiens,PMID:23637823,"{""frequency"": """", ""identification"": ""multimer-...","{""cell.subset"": ""CD8+"", ""clone.id"": ""4A"", ""don...","{""cdr3"": ""CAMSLYYGGSQGNLIF"", ""cdr3_old"": ""CAMS...",2
74807,24876,TRB,CASSLEIFGGIADTDTQYF,TRBV11-2*01,TRBJ2-3*01,HomoSapiens,HLA-A*02,B2M,MHCI,KIFGSLAFL,ERBB2,HomoSapiens,PMID:23637823,"{""frequency"": """", ""identification"": ""multimer-...","{""cell.subset"": ""CD8+"", ""clone.id"": ""4A"", ""don...","{""cdr3"": ""CASSLEIFGGIADTDTQYF"", ""cdr3_old"": ""C...",2
74844,24877,TRA,CAASIGDFGNEKLTF,TRAV13-1*01,TRAJ48*01,HomoSapiens,HLA-B*07:02,B2M,MHCI,RPHERNGFTVL,pp65,CMV,PMID:22323539,"{""frequency"": ""3/3"", ""identification"": ""antige...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CAASIGDFGNEKLTF"", ""cdr3_old"": ""CAASI...",1
